<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Definitions" data-toc-modified-id="Definitions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Definitions</a></span><ul class="toc-item"><li><span><a href="#Application:-least-squares" data-toc-modified-id="Application:-least-squares-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Application: least squares</a></span></li></ul></li><li><span><a href="#Gram-Schmidt-procedure" data-toc-modified-id="Gram-Schmidt-procedure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gram-Schmidt procedure</a></span><ul class="toc-item"><li><span><a href="#GS-conducts-reduced-QR" data-toc-modified-id="GS-conducts-reduced-QR-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>GS conducts reduced QR</a></span></li><li><span><a href="#Classical-Gram-Schmidt" data-toc-modified-id="Classical-Gram-Schmidt-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Classical Gram-Schmidt</a></span></li><li><span><a href="#Modified-Gram-Schmidt" data-toc-modified-id="Modified-Gram-Schmidt-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Modified Gram-Schmidt</a></span></li></ul></li><li><span><a href="#QR-by-Householder-transform" data-toc-modified-id="QR-by-Householder-transform-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>QR by Householder transform</a></span><ul class="toc-item"><li><span><a href="#Algorithm" data-toc-modified-id="Algorithm-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Algorithm</a></span></li><li><span><a href="#Householder-QR-with-column-pivoting" data-toc-modified-id="Householder-QR-with-column-pivoting-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Householder QR with column pivoting</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Implementation</a></span></li></ul></li><li><span><a href="#QR-by-Givens-rotation" data-toc-modified-id="QR-by-Givens-rotation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>QR by Givens rotation</a></span></li><li><span><a href="#Applications" data-toc-modified-id="Applications-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Applications</a></span><ul class="toc-item"><li><span><a href="#Linear-regression" data-toc-modified-id="Linear-regression-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Linear regression</a></span></li></ul></li><li><span><a href="#Further-reading" data-toc-modified-id="Further-reading-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Further reading</a></span></li><li><span><a href="#Acknowledgment" data-toc-modified-id="Acknowledgment-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Acknowledgment</a></span></li></ul></div>

In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-10700K CPU @ 3.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [2]:
using Pkg
Pkg.activate("../..")
Pkg.status()

  Activating environment at `~/opt/Project.toml`


      Status `~/opt/Project.toml`
  [6e4b80f9] BenchmarkTools v1.2.0
  [f65535da] Convex v0.14.16
  [a93c6f00] DataFrames v1.2.2
  [31c24e10] Distributions v0.24.18
  [c91e804a] Gadfly v1.3.3
  [bd48cda9] GraphRecipes v0.5.7
  [916415d5] Images v0.24.1
  [d330b81b] PyPlot v2.10.0
  [6f49c342] RCall v0.13.12
  [2f01184e] SparseArrays
  [10745b16] Statistics


# QR Decomposition

`QR` -> 가장 기본이 되는 방법

* We learned Cholesky decomposition as **one** approach for solving linear regression.
    
    촐레스키 분해가 가장 스탠다드한 방법은 아니다
    

* Another approach for linear regression uses the QR decomposition.  
    **This is how the `lm()` function in R does linear regression.**
    \
    **This is also how Julia's (and MATLAB's) `\` works for rectangular matrices.**
    
직사각형 행렬이면 least square 해를 찾는다.

In [4]:
using Random

Random.seed!(280) # seed

n, p = 5, 3
X = randn(n, p) # predictor matrix
y = randn(n)    # response vector

# backslash finds the (minimum L2 norm) least squares solution
X \ y    # NOT X^{-1}y or solve X*beta=y, BUT minimize norm(y - X * beta)^2 (least squares)

3-element Vector{Float64}:
 0.37954666766986195
 0.6508866456093484
 0.3922504195653549

We want to understand what is QR and how it is used for solving least squares problem.

## Definitions

* Assume $\mathbf{X} \in \mathbb{R}^{n \times p}$ has full column rank(행들이 선형독립). Necessarilly $n \ge p$.


* **Full QR decomposition**:  
$$
    \mathbf{X} = \mathbf{Q} \mathbf{R},  
$$
where  
    - $\mathbf{Q} \in \mathbb{R}^{n \times n}$, $\mathbf{Q}^T \mathbf{Q} = \mathbf{Q}\mathbf{Q}^T = \mathbf{I}_n$. In other words, $\mathbf{Q}$ is an orthogonal matrix.  
        - First $p$ columns of $\mathbf{Q}$ form an orthonormal basis of ${\cal R}(\mathbf{X})$ (**range** or column space of $\mathbf{X}$)      
        - Last $n-p$ columns of $\mathbf{Q}$ form an orthonormal basis of ${\cal N}(\mathbf{X}^T)$ (**null space** of $\mathbf{X}^T$)
        - Recall that $\mathcal{N}(\mathbf{X}^T)=\mathcal{R}(\mathbf{X})^{\perp}$ and $\mathcal{R}(\mathbf{X}) \oplus \mathcal{N}(\mathbf{X}^T) = \mathbb{R}^n$.
    - $\mathbf{R} \in \mathbb{R}^{n \times p}$  is upper triangular with positive diagonal entries. 
        - The lower $(n-p)\times p$ block of $\mathbf{R}$ is $\mathbf{0}$ (why?).

In [ ]:
# Q : 직교행렬

 * **Reduced QR decomposition**:
$$
    \mathbf{X} = \mathbf{Q}_1 \mathbf{R}_1,
$$
where
    - $\mathbf{Q}_1 \in \mathbb{R}^{n \times p}$, $\mathbf{Q}_1^T \mathbf{Q}_1 = \mathbf{I}_p$. In other words, $\mathbf{Q}_1$ is a partially orthogonal matrix(모든 columns 독립, Full column matrix). Note $\mathbf{Q}_1\mathbf{Q}_1^T \neq \mathbf{I}_n$.
    - $\mathbf{R}_1 \in \mathbb{R}^{p \times p}$  is an upper triangular matrix with positive diagonal entries.(rank($\mathbf{R}_1$) = p, full rank, invertible)
    - $\mathbf{Q}$ from Full QR : $\mathbf{Q} = [\mathbf{Q}_1, \mathbf{Q}_2]$; $\mathbf{R} = \begin{bmatrix} \mathbf{R}_1 \\ \mathbf{0} \end{bmatrix}$

* Given QR decomposition $\mathbf{X} = \mathbf{Q} \mathbf{R}$,
    $$
    \mathbf{X}^T \mathbf{X} = \mathbf{R}^T \mathbf{Q}^T \mathbf{Q} \mathbf{R} = \mathbf{R}^T \mathbf{R} = \mathbf{R}_1^T \mathbf{R}_1.
    $$
    - Once we have a (reduced) QR decomposition of $\mathbf{X}$, we automatically have the Cholesky decomposition of the *Gram matrix*  $\mathbf{X}^T \mathbf{X}$.
    
    
QR 알고 있으면, Cholesky는 저절로 얻게 되는 것

### Application: least squares

* Normal equation
$$
    \mathbf{X}^T\mathbf{X}\beta = \mathbf{X}^T\mathbf{y}
$$
is equivalently written with reduced QR as
$$
    \mathbf{R}_1^T\mathbf{R}_1\beta = \mathbf{R}_1^T\mathbf{Q}_1^T\mathbf{y}
$$


* Since $\mathbf{R}_1$ is invertible, we only need to solve the triangluar system
$$
    \mathbf{R}_1\beta = \mathbf{Q}_1^T\mathbf{y}
$$
Multiplication $\mathbf{Q}_1^T \mathbf{y}$ is done implicitly (see below).


* This method is numerically more stable than directly solving the normal equation, since $\kappa(\mathbf{X}^T\mathbf{X}) = \kappa(\mathbf{X})^2$! (QR로 풀면 $\kappa(\mathbf{X})$)


* In case we need standard errors, compute inverse of $\mathbf{R}_1^T \mathbf{R}_1$. This involves triangular solves.

## Gram-Schmidt procedure

* Wait! Does $\mathbf{X}$ always have a QR decomposition?
    - Yes. It is equivalent to the Gram-Schmidt procedure for basis orthonormalization.

<img src="https://upload.wikimedia.org/wikipedia/commons/e/e7/Jørgen_Pedersen_Gram_by_Johannes_Hauerslev.jpg" width="200" align="center"/>

[Jørgen Pedersen Gram, 1850-1916](https://en.wikipedia.org/wiki/Jørgen_Pedersen_Gram)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Erhard_Schmidt.jpg/220px-Erhard_Schmidt.jpg" width="200" align="center"/>

[Erhard Schmidt, 1876-1959](https://en.wikipedia.org/wiki/Erhard_Schmidt)

QR decomposition 과 Gram-Schmidt 과정은 완전히 일치하는 과정

* Assume $\mathbf{X} = [\mathbf{x}_1 | \dotsb | \mathbf{x}_p] \in \mathbb{R}^{n \times p}$ has full column rank. That is,  $\mathbf{x}_1,\ldots,\mathbf{x}_p$ are *linearly independent*.


* Gram-Schmidt (GS) procedure produces nested orthonormal basis vectors $\{\mathbf{q}_1, \dotsc, \mathbf{q}_p\}$ that spans $\mathcal{R}(\mathbf{X})$, i.e.,
$$
\begin{split}
    \text{span}(\{\mathbf{x}_1\}) &= \text{span}(\{\mathbf{q}_1\}) \\
    \text{span}(\{\mathbf{x}_1, \mathbf{x}_2\}) &= \text{span}(\{\mathbf{q}_1, \mathbf{q}_2\}) \\
    & \vdots \\
    \text{span}(\{\mathbf{x}_1, \mathbf{x}_2, \dotsc, \mathbf{x}_p\}) &= \text{span}(\{\mathbf{q}_1, \mathbf{q}_2, \dotsc, \mathbf{q}_p\}) 
\end{split}
$$
and $\langle \mathbf{q}_i, \mathbf{q}_j \rangle = \delta_{ij} = \begin{cases}1 & i=j\\0 & i\neq j\end{cases}$. 

* The algorithm:
    0. Initialize $\mathbf{q}_1 = \mathbf{x}_1 / \|\mathbf{x}_1\|_2$
    0. For $k=2, \ldots, p$, 
$$
\begin{eqnarray*}
	\mathbf{v}_k &=& \mathbf{x}_k - P_{\text{span}(\{\mathbf{q}_1,\ldots,\mathbf{q}_{k-1}\})}(\mathbf{x}_k) = \mathbf{x}_k -  \sum_{j=1}^{k-1} \langle \mathbf{q}_j, \mathbf{x}_k \rangle \cdot \mathbf{q}_j \\
	\mathbf{q}_k &=& \mathbf{v}_k / \|\mathbf{v}_k\|_2
\end{eqnarray*}
$$


$$
\begin{align*}
    v_i &: \text{orthogonal basis}\\
    q_i &: \text{orthonormal basis}
\end{align*}
$$




$\langle \mathbf{q}_j, \mathbf{x}_k \rangle$ : $q_j$를 $x_k$로 사영한 벡터의 크기

### GS conducts reduced QR

* $\mathbf{Q} = [\mathbf{q}_1 | \dotsb | \mathbf{q}_p] = \mathbf{Q}_1$ in Full QR. Obviously $\mathbf{Q}^T \mathbf{Q} = \mathbf{I}_p$.

그램슈미트 이용하면, $Q_1$구할 수 ㅇ

* Where is $\mathbf{R}$? 
    - Let $r_{jk} = \langle \mathbf{q}_j, \mathbf{x}_k \rangle$ for $j < k$, and $r_{kk} = \|\mathbf{v}_k\|_2$.
    - Re-write the above expression:
$$
    r_{kk} \mathbf{q}_k = \mathbf{x}_k -  \sum_{j=1}^{k-1} r_{jk} \cdot \mathbf{q}_j
$$

$$$$

        or
$$
    \mathbf{x}_k = r_{kk} \mathbf{q}_k +  \sum_{j=1}^{k-1} r_{jk} \cdot \mathbf{q}_j
    .
$$

$$X = Q_1 R_1 \text{과 비슷한 과정이라고 생각}$$


<br>
<br>


   - If we let $r_{jk} = 0$ for $j > k$, then $\mathbf{R}=(r_{jk})$ is upper triangular and
   
$$
    \mathbf{X} = \mathbf{Q}\mathbf{R}
    .
$$
 
<img src="https://dsc-spidal.github.io/harp/img/daalAlgosNew/HarpIllustrations_QR.png" width="800" align="center"/> 
  
Source: <https://dsc-spidal.github.io/harp/docs/harpdaal/algorithms/>  
  

### Classical Gram-Schmidt

In [1]:
using LinearAlgebra
function cgs(X::Matrix{T}) where T<:AbstractFloat
    n, p = size(X)
    Q = Matrix{T}(undef, n, p)
    R = zeros(T, p, p)
    for j=1:p
        Q[:, j] .= X[:, j]   # v_k 만들 때 x_k 에서 계속 빼줘야하기 때문.
        for i=1:j-1
            R[i, j] = dot(Q[:, i], X[:, j])
            Q[:, j] .-= R[i, j] * Q[:, i]    # proj한 벡터들 계속 빼주는 것 생각
        end
        R[j, j] = norm(Q[:, j])
        Q[:, j] /= R[j, j]
    end
    Q, R
end

cgs (generic function with 1 method)

* CGS is *unstable* (we lose orthogonality due to roundoff errors) when columns of $\mathbf{X}$ are almost collinear.

수학적으론 열들이 선형독립이지만, 수치적으로 선형 종속인 열들이 존재한다면 계산이 제대로 이루어지지 않는다.

floating point를 사용하기 때문에 0을 만들어야 하는 orthogonality과정에서 오차가 발생하기 쉽다.

In [31]:
e = eps(Float32)
A = [1f0 1f0 1f0; e 0 0; 0 e 0; 0 0 e]
# machine epsilong 발생

4×3 Matrix{Float32}:
 1.0         1.0         1.0
 1.19209f-7  0.0         0.0
 0.0         1.19209f-7  0.0
 0.0         0.0         1.19209f-7

In [32]:
A[:,1]/norm(A[:,1])

4-element Vector{Float32}:
 1.0
 1.1920929f-7
 0.0
 0.0

In [33]:
Q, R = cgs(A)
Q
# 4×3 Matrix{Float32}:
#  1.0          0.0        0.0
#  1.19209f-7  -0.707107  -0.408248
#  0.0          0.707107  -0.408248
#  0.0          0.0        0.816497

4×3 Matrix{Float32}:
 1.0          0.0        0.0
 1.19209f-7  -0.707107  -0.707107
 0.0          0.707107   0.0
 0.0          0.0        0.707107

In [14]:
R
# 3×3 Matrix{Float64}:
#  4.89898  -8.57321   7.75672
#  0.0       4.94975  -0.303046
#  0.0       0.0       1.31966

3×3 Matrix{Float64}:
 4.89898  -8.57321   7.75672
 0.0       4.94975  -0.303046
 0.0       0.0       1.31966

In [5]:
transpose(Q)*Q

3×3 Matrix{Float32}:
  1.0         -8.42937f-8  -8.42937f-8
 -8.42937f-8   1.0          0.5
 -8.42937f-8   0.5          1.0

* `Q` is hardly orthogonal.
* Where exactly does the problem occur? (HW)

### Modified Gram-Schmidt

* The algorithm:
    0. Initialize $\mathbf{q}_1 = \mathbf{x}_1 / \|\mathbf{x}_1\|_2$
    0. For $k=2, \ldots, p$, 
$$
\begin{eqnarray*}
	\mathbf{v}_k &=& \mathbf{x}_k - P_{\text{span}(\{\mathbf{q}_1,\ldots,\mathbf{q}_{k-1}\})}(\mathbf{x}_k) = \mathbf{x}_k -  \sum_{j=1}^{k-1} \langle \mathbf{q}_j, \mathbf{x}_k \rangle \cdot \mathbf{q}_j \\
    &=&  \mathbf{x}_k -  \sum_{j=1}^{k-1} \left\langle \mathbf{q}_j, \mathbf{x}_k - \sum_{l=1}^{j-1}\langle \mathbf{q}_l, \mathbf{x}_k \rangle \mathbf{q}_l \right\rangle \cdot \mathbf{q}_j \\
	\mathbf{q}_k &=& \mathbf{v}_k / \|\mathbf{v}_k\|_2
\end{eqnarray*}
$$

In [6]:
function mgs!(X::Matrix{T}) where T<:AbstractFloat
    n, p = size(X)
    R = zeros(T, p, p)
    for j=1:p
        for i=1:j-1
            R[i, j] = dot(X[:, i], X[:, j])
            X[:, j] -= R[i, j] * X[:, i]
        end
        R[j, j] = norm(X[:, j])
        X[:, j] /= R[j, j]
    end
    X, R
end

# function cgs(X::Matrix{T}) where T<:AbstractFloat
#     n, p = size(X)
#     Q = Matrix{T}(undef, n, p)
#     R = zeros(T, p, p)
#     for j=1:p
#         Q[:, j] .= X[:, j]   # v_k 만들 때 x_k 에서 계속 빼줘야하기 때문.
#         for i=1:j-1
#             R[i, j] = dot(Q[:, i], X[:, j])
#             Q[:, j] .-= R[i, j] * Q[:, i]    # proj한 벡터들 계속 빼주는 것 생각
#         end
#         R[j, j] = norm(Q[:, j])
#         Q[:, j] /= R[j, j]
#     end
#     Q, R
# end

mgs! (generic function with 1 method)

* $\mathbf{X}$ is overwritten by $\mathbf{Q}$ and $\mathbf{R}$ is stored in a separate array.

In [34]:
Q, R = mgs!(copy(A))
Q

4×3 Matrix{Float32}:
 1.0          0.0        0.0
 1.19209f-7  -0.707107  -0.408248
 0.0          0.707107  -0.408248
 0.0          0.0        0.816497

In [35]:
R

3×3 Matrix{Float32}:
 1.0  1.0         1.0
 0.0  1.68587f-7  8.42937f-8
 0.0  0.0         1.46001f-7

In [17]:
transpose(Q)*Q

3×3 Matrix{Float64}:
  1.0           3.88578e-16  -1.25594e-15
  3.88578e-16   1.0          -1.38778e-17
 -1.25594e-15  -1.38778e-17   1.0

* So MGS is more stable than CGS. However, even MGS is not completely immune to instability.

In [18]:
B = [0.7f0 0.7071068f0; 0.7000001f0 0.7071068f0]

2×2 Matrix{Float32}:
 0.7  0.707107
 0.7  0.707107

In [19]:
Q, R = mgs!(copy(B))
Q

2×2 Matrix{Float32}:
 0.707107  1.0
 0.707107  0.0

In [20]:
transpose(Q)*Q

2×2 Matrix{Float32}:
 1.0       0.707107
 0.707107  1.0

* `Q` is hardly orthogonal.
* Where exactly the problem occurs? (HW)

* Computational cost of CGS and MGS is $\sum_{k=1}^p 4n(k-1) \approx 2np^2$.

* There are 3 algorithms to compute QR: (modified) Gram-Schmidt, Householder transform, (fast) Givens transform.

    In particular, the **Householder transform** for QR is implemented in LAPACK and thus used in R and Julia.

## QR by Householder transform

<img src="http://www-history.mcs.st-andrews.ac.uk/BigPictures/Householder_2.jpeg" width="200" align="center"/>

[Alston Scott Householder (1904-1993)](https://en.wikipedia.org/wiki/Alston_Scott_Householder)

* **This is the algorithm for solving linear regression in R**.

* Assume again $\mathbf{X} = [\mathbf{x}_1 | \dotsb | \mathbf{x}_p] \in \mathbb{R}^{n \times p}$ has full column rank.

* Gram-Schmidt can be understood as:
$$
    \mathbf{X}\mathbf{R}_{1} \mathbf{R}_2 \cdots  \mathbf{R}_n = \mathbf{Q}_1
$$
where $\mathbf{R}_j$ are a sequence of upper triangular matrices.

* Householder QR does
$$
    \mathbf{H}_{p} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{X} = \begin{pmatrix} \mathbf{R}_1 \\ \mathbf{0} \end{pmatrix},
$$
where $\mathbf{H}_j \in \mathbf{R}^{n \times n}$ are a sequence of Householder transformation matrices.

    It yields the **full QR** where $\mathbf{Q} = \mathbf{H}_1 \cdots \mathbf{H}_p \in \mathbb{R}^{n \times n}$. Recall that CGS/MGS only produces the **reduced QR** decomposition.

$H_i$를 반복해서 곱해도 orthogonality 가 보존되기 때문에, 수치적으로 유리하다

* For arbitrary $\mathbf{v}, \mathbf{w} \in \mathbb{R}^{n}$ with $\|\mathbf{v}\|_2 = \|\mathbf{w}\|_2$, we can construct a **Householder matrix** (or **Householder reflector**)
$$
    \mathbf{H} = \mathbf{I}_n - 2 \mathbf{u} \mathbf{u}^T, \quad \mathbf{u} = \frac{1}{\|\mathbf{v} - \mathbf{w}\|_2} (\mathbf{v} - \mathbf{w}),
$$
that transforms $\mathbf{v}$ to $\mathbf{w}$:
$$
	\mathbf{H} \mathbf{v} = \mathbf{w}.
$$
$\mathbf{H}$ is symmetric and orthogonal. Calculation of Householder vector $\mathbf{u}$ costs $4n$ flops for general $\mathbf{u}$ and $\mathbf{w}$.

<img src="https://www.cs.utexas.edu/users/flame/laff/alaff-beta/images/Chapter03/reflector.png" width="400" align="center"/>

Red arrow : v, Blue arrow : w 에 대입해서 생각

u벡터를 normal vector로 갖는 hyperplane상에서 x, Hx 는 서로 대칭인 관계이다.

Source: <https://www.cs.utexas.edu/users/flame/laff/alaff-beta/images/Chapter03/reflector.png>

In [ ]:
# 단위벡터 u가 존재하면, 이에 대응하는 Householder matrix를 만들 수 있다.

* Now choose $\mathbf{H}_1$ so that
$$
	\mathbf{H}_1 \mathbf{x}_1 = \begin{pmatrix} \|\mathbf{x}_{1}\|_2 \\ 0 \\ \vdots \\ 0 \end{pmatrix}.
$$
That is, $\mathbf{v} = \mathbf{x}_1$ and $\mathbf{w} = \|\mathbf{x}_1\|_2\mathbf{e}_1$.

* Left-multiplying $\mathbf{H}_1$ zeros out the first column of $\mathbf{X}$ below (1, 1).

$$
\mathbf{H}_1\mathbf{X} = 
\begin{bmatrix} 
\times & \times & \times & \times \\ 
0 & \boldsymbol{\times} & \boldsymbol{\times} & \boldsymbol{\times}  \\
0 & \times & \times & \times  \\
0 & \times & \times & \times  \\
0 & \times & \times & \times 
\end{bmatrix} 
=
\begin{bmatrix}
m_{11} & m_{12} & m_{13} & m_{14} \\
\mathbf{0} & \tilde{\bf x}_2 & \tilde{\bf x}_3 & \tilde{\bf x}_4
\end{bmatrix} 
$$

* Take $\mathbf{H}_2$ to zero the second column below diagonal: i.e., choose $\tilde{\bf H}_2 \in \mathbb{R}^{(n-1)\times(n-1)}$ such that

$$
\tilde{\bf H}_2 \tilde{\bf x}_2 = \begin{bmatrix} \Vert \tilde{\bf x}_2 \Vert_2 \\ 0 \\ \vdots \\ 0 \end{bmatrix}
$$

$$
\mathbf{H}_2 = \begin{bmatrix} 1 & \mathbf{0}^T \\ \mathbf{0} & \tilde{\bf H}_2 \end{bmatrix} \in \mathbb{R}^{n \times n}
$$

$$
\mathbf{H}_2\mathbf{H}_1\mathbf{X} = 
\begin{bmatrix} 
\times & \times & \times & \times \\ 
0 & \boldsymbol{\times} & \boldsymbol{\times} & \boldsymbol{\times}  \\
0 & \mathbf{0} & \boldsymbol{\times} & \boldsymbol{\times}  \\
0 & \mathbf{0} & \boldsymbol{\times} & \boldsymbol{\times}  \\
0 & \mathbf{0} & \boldsymbol{\times} & \boldsymbol{\times} 
\end{bmatrix} 
$$

* In general, choose the $j$-th Householder transform $\mathbf{H}_j = \mathbf{I}_n - 2 \mathbf{u}_j \mathbf{u}_j^T$, where 

$$
     \mathbf{u}_j = \begin{bmatrix} \mathbf{0}_{j-1} \\ {\tilde u}_j \end{bmatrix}, \quad {\tilde u}_j \in \mathbb{R}^{n-j+1},
$$

to zero the $j$-th column below diagonal. $\mathbf{H}_j$ takes the form

$$
	\mathbf{H}_j = \begin{bmatrix}
	\mathbf{I}_{j-1} & \\
	& \mathbf{I}_{n-j+1} - 2 {\tilde u}_j {\tilde u}_j^T
	\end{bmatrix} = \begin{bmatrix}
	\mathbf{I}_{j-1} & \\
	& {\tilde H}_{j}
	\end{bmatrix}.
$$

* Applying a Householder transform $\mathbf{H} = \mathbf{I} - 2 \mathbf{u} \mathbf{u}^T$ to a matrix $\mathbf{X} \in \mathbb{R}^{n \times p}$

$$
	\mathbf{H} \mathbf{X} = \mathbf{X} - 2 \mathbf{u} (\mathbf{u}^T \mathbf{X})
$$

좌변만 보면 $n^2p$ flops일 것 같지만, 오른쪽 계산을 살펴보면 $4np$

costs $4np$ flops. **Householder updates never entails explicit formation of the Householder matrices.**


* Note applying ${\tilde H}_j$ to $\mathbf{X}$ only needs $4(n-j+1)(p-j+1)$ flops. (j가 커지면서, $\tilde{H}_j$의 차원(n-j+1)도 줄어들고, 0벡터가 포함되지 않은 열(p-j+1)도 줄어들기 때문에 flop이 줄어든다)

### Algorithm

```Julia
for j=1:p
    u = House!(X[j:n, j])
    for i=j:p
        X[j:n, j:p] .-= 2u*(u'X[j:n, j:p])
    end
end
```

* The process is done in place. Upper triangular part of $\mathbf{X}$ is overwritten by $\mathbf{R}_1$ and the essential Householder vectors ($\tilde u_{j1}$ is normalized to 1) are stored in $\mathbf{X}[j:n,j]$.


* At $j$-th stage
     0. computing the Householder vector ${\tilde u}_j$ costs $3(n-j+1)$ flops
     0. applying the Householder transform ${\tilde H}_j$ to the $\mathbf{X}[j:n, j:p]$ block costs $4(n-j+1)(p-j+1)$ flops  


* In total we need $\sum_{j=1}^p [3(n-j+1) + 4(n-j+1)(p-j+1)] \approx 2np^2 - \frac 23 p^3$ flops.

* Where is $\mathbf{Q}$? 
    - $\mathbf{Q} = \mathbf{H}_1 \cdots \mathbf{H}_p$. In some applications, it's necessary to form the orthogonal matrix $\mathbf{Q}$. 
    
Q를 얻고 싶으면, u로 부터 다시 만들어야 한다.

Accumulating $\mathbf{Q}$ costs another $2np^2 - \frac 23 p^3$ flops.


* When computing $\mathbf{Q}^T \mathbf{v}$ or $\mathbf{Q} \mathbf{v}$ as in some applications (e.g., solve linear equation using QR), no need to form $\mathbf{Q}$. Simply apply Householder transforms successively to the vector $\mathbf{v}$. (HW)


* Computational cost of Householder QR for linear regression: $2n p^2 - \frac 23 p^3$ (regression coefficients and $\hat \sigma^2$) or more (fitted values, s.e., ...).

In [ ]:
# 

### Householder QR with column pivoting

X가 full rank가 아닌 경우, column pivoting

Consider rank deficient $\mathbf{X}$.

* At the $j$-th stage, swap the column `X[:, j]` with `X[:, k]`(j<k) where `k` is the column number in `X[j:n,j:p]` with maximum $\ell_2$ norm to be the pivot column. If the maximum $\ell_2$ norm is 0, it stops, ending with (maximum norm이 0이 되는 순간에 문제가 발생한다.)

$$
\mathbf{X} \mathbf{P} = \mathbf{Q} \begin{bmatrix} \mathbf{R}_{11} & \mathbf{R}_{12} \\ \mathbf{0}_{(n-r) \times r} & \mathbf{0}_{(n-r) \times (p-r)} \end{bmatrix},
$$

where $\mathbf{P} \in \mathbb{R}^{p \times p}$ is a permutation matrix and $r$ is the rank of $\mathbf{X}$. QR with column pivoting is rank revealing.

### Implementation

* Julia functions: [`qr`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.qr), [`qrfact!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.qr!), or call LAPACK wrapper functions [`geqrf!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.geqrf!) and [`geqp3!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.geqp3!)

* R function: [`qr`](https://stat.ethz.ch/R-manual/R-devel/library/base/html/qr.html). Wraps LAPACK routine [`dgeqp3`](http://www.netlib.org/lapack/explore-html/dd/d9a/group__double_g_ecomputational_ga1b0500f49e03d2771b797c6e88adabbb.html)  (with `LAPACK=TRUE`; default uses LINPACK, an ancient version of LAPACK).

In [140]:
X, y

([0.12623784496408763 -1.1278278196026448 -0.8826696457022515; -2.3468794813834966 1.1478628422667982 1.7138424693341203; … ; -0.23920888512829233 -0.23706547458394342 1.0818212935057896; -0.5784508270929073 -0.6809935026697 -0.17040614729185025], [-1.794259674143309, 1.0913793110025305, 0.4266277597108536, -0.6244337204329091, 0.03204861737738283])

In [141]:
X \ y # least squares solution by QR

3-element Vector{Float64}:
 0.37954666766986195
 0.6508866456093484
 0.3922504195653549

In [136]:
# same as
qr(X) \ y

3-element Vector{Float64}:
 0.37954666766986195
 0.6508866456093482
 0.3922504195653549

In [137]:
cholesky(X'X) \ (X'y) # least squares solution by Cholesky

3-element Vector{Float64}:
 0.37954666766986256
 0.6508866456093485
 0.3922504195653555

In [142]:
# QR factorization with column pivoting
xqr = qr(X, Val(true))

# full rank 이므로 pivoting은 일어나지 않았다.

QRPivoted{Float64, Matrix{Float64}}
Q factor:
5×5 LinearAlgebra.QRPackedQ{Float64, Matrix{Float64}}:
 -0.0407665  -0.692007    0.318693   0.185526  -0.619257
  0.757887   -0.0465712  -0.260086  -0.522259  -0.288166
 -0.618938   -0.233814   -0.404293  -0.624592  -0.0931611
  0.0772486  -0.235405   -0.808135   0.53435    0.00216687
  0.186801   -0.639431    0.119392  -0.131075   0.724429
R factor:
3×3 Matrix{Float64}:
 -3.09661  1.60888   1.84089
  0.0      1.53501   0.556903
  0.0      0.0      -1.32492
permutation:
3-element Vector{Int64}:
 1
 2
 3

In [139]:
xqr \ y # least squares solution

3-element Vector{Float64}:
 0.37954666766986195
 0.6508866456093484
 0.3922504195653549

In [133]:
# thin Q matrix multiplication (a sequence of Householder transforms)
norm(xqr.Q * xqr.R - X[:, xqr.p]) # recovers X (with columns permuted)

1.191228434442578e-15

## QR by Givens rotation

* Householder transform $\mathbf{H}_j$ introduces batch of zeros into a vector.

* **Givens transform** (aka **Givens rotation**, **Jacobi rotation**, **plane rotation**) selectively zeros one element of a vector.

* Overall QR by Givens rotation is less efficient than the Householder method, but is better suited for matrices with structured patterns of nonzero elements.

* **Givens/Jacobi rotations**: 
$$
	\mathbf{G}(i,k,\theta) = \begin{bmatrix} 
	1 & & 0 & & 0 & & 0 \\
	\vdots & \ddots & \vdots & & \vdots & & \vdots \\
	0 & & c & & s & & 0 \\ 
	\vdots & & \vdots & \ddots & \vdots & & \vdots \\
	0 & & - s & & c & & 0 \\
	\vdots & & \vdots & & \vdots & \ddots & \vdots \\
	0 & & 0 & & 0 & & 1 \end{bmatrix},
$$
where $c = \cos(\theta)$ and $s = \sin(\theta)$. $\mathbf{G}(i,k,\theta)$ is orthogonal.

* Pre-multiplication by $\mathbf{G}(i,k,\theta)^T$ rotates counterclockwise $\theta$ radians in the $(i,k)$ coordinate plane. If $\mathbf{x} \in \mathbb{R}^n$ and $\mathbf{y} = \mathbf{G}(i,k,\theta)^T \mathbf{x}$, then
$$
	y_j = \begin{cases}
	cx_i - s x_k & j = i \\
	sx_i + cx_k & j = k \\
	x_j & j \ne i, k
	\end{cases}.
$$
Apparently if we choose $\tan(\theta) = -x_k / x_i$, or equivalently,
$$
\begin{eqnarray*}
	c = \frac{x_i}{\sqrt{x_i^2 + x_k^2}}, \quad s = \frac{-x_k}{\sqrt{x_i^2 + x_k^2}},
\end{eqnarray*}
$$
then $y_k=0$.

* Pre-applying Givens transform $\mathbf{G}(i,k,\theta)^T \in \mathbb{R}^{n \times n}$ to a matrix $\mathbf{A} \in \mathbb{R}^{n \times m}$ only effects two rows of $\mathbf{
A}$:
$$
	\mathbf{A}([i, k], :) \gets \begin{bmatrix} c & s \\ -s & c \end{bmatrix}^T \mathbf{A}([i, k], :),
$$
costing $6m$ flops.

* Post-applying Givens transform $\mathbf{G}(i,k,\theta) \in \mathbb{R}^{m \times m}$ to a matrix $\mathbf{A} \in \mathbb{R}^{n \times m}$ only effects two columns of $\mathbf{A}$:
$$
	\mathbf{A}(:, [i,k]) \gets \mathbf{A}(:, [i,k]) \begin{bmatrix} c & s \\ -s & c \end{bmatrix},
$$
costing $6n$ flops.

* QR by Givens: $\mathbf{G}_t^T \cdots \mathbf{G}_1^T \mathbf{X} =  \begin{bmatrix} \mathbf{R}_1 \\ \mathbf{0} \end{bmatrix}$.

<img src="QR_by_Givens.png" width="600" align="center"/>

* Zeros in $\mathbf{X}$ can also be introduced row-by-row.

* If $\mathbf{X} \in \mathbb{R}^{n \times p}$, the total cost is $3np^2 - p^3$ flops and $O(np)$ square roots.

* Note each Givens transform can be summarized by a single number, which is stored in the zeroed entry of $\mathbf{X}$.

## Applications

### Linear regression

* QR decomposition of $\mathbf{X}$: $2np^2 - \frac 23 p^3$ flops.

* Solve $\mathbf{R}^T \mathbf{R} \beta = \mathbf{R}^T \mathbf{Q}^T \mathbf{y}$ for $\beta$.

* If $\mathbf{X}$ is full rank, then $\mathbf{R}$ is invertible, so we only need to solve the triangular system
$$
    \mathbf{R} \beta = \mathbf{Q}^T \mathbf{y}
    .
$$
Multiplication $\mathbf{Q}^T \mathbf{y}$ is done implicitly.

* If need standard errors, compute inverse of $\mathbf{R}^T \mathbf{R}$. This involves triangular solves.

## Further reading

* Section II.5.3 of [Computational Statistics](https://link.springer.com/book/10.1007%2F978-0-387-98144-4) by James Gentle (2010).

* Chapter 5 of [Matrix Computation](https://www.amazon.com/Computations-Hopkins-Studies-Mathematical-Sciences/dp/1421407949/ref=sr_1_1?keywords=matrix+computation+golub&qid=1567157884&s=gateway&sr=8-1) by Gene Golub and Charles Van Loan (2013).

## Acknowledgment

Many parts of this lecture note is based on [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.